In [1]:
# Import libraries
import datetime as dt
import json
import requests as rq
#import bs4 as bs
#from selenium import webdriver
import pandas as pd
import numpy as np
#import pybit

Достаю для начала все доступные на данный момент даты исполнения и страйки:

In [2]:
key_symbols='https://api-testnet.bybit.com/option/usdc/openapi/public/v1/symbols'
data=rq.get(key_symbols).json()
dataList=[] #достаю словари с необходимым полем symbol
for i in data['result']['dataList']:
                dataList.append(i)           

In [3]:
symbols=[]
for j in dataList:
    symbols.append(j['symbol'])

Достаю названия доступных на данный момент опционов:

In [4]:
symbols

['BTC-30DEC22-300000-C',
 'BTC-30DEC22-300000-P',
 'BTC-30DEC22-250000-C',
 'BTC-30DEC22-250000-P',
 'BTC-30DEC22-200000-C',
 'BTC-30DEC22-200000-P',
 'BTC-30DEC22-150000-C',
 'BTC-30DEC22-150000-P',
 'BTC-30DEC22-120000-C',
 'BTC-30DEC22-120000-P',
 'BTC-30DEC22-100000-C',
 'BTC-30DEC22-100000-P',
 'BTC-30DEC22-90000-C',
 'BTC-30DEC22-90000-P',
 'BTC-30DEC22-80000-C',
 'BTC-30DEC22-80000-P',
 'BTC-30DEC22-70000-C',
 'BTC-30DEC22-70000-P',
 'BTC-30DEC22-60000-C',
 'BTC-30DEC22-60000-P',
 'BTC-30DEC22-55000-C',
 'BTC-30DEC22-55000-P',
 'BTC-30DEC22-50000-C',
 'BTC-30DEC22-50000-P',
 'BTC-30DEC22-45000-C',
 'BTC-30DEC22-45000-P',
 'BTC-30DEC22-40000-C',
 'BTC-30DEC22-40000-P',
 'BTC-30DEC22-38000-C',
 'BTC-30DEC22-38000-P',
 'BTC-30DEC22-36000-C',
 'BTC-30DEC22-36000-P',
 'BTC-30DEC22-35000-C',
 'BTC-30DEC22-35000-P',
 'BTC-30DEC22-34000-C',
 'BTC-30DEC22-34000-P',
 'BTC-30DEC22-32000-C',
 'BTC-30DEC22-32000-P',
 'BTC-30DEC22-30000-C',
 'BTC-30DEC22-30000-P',
 'BTC-30DEC22-28000-C',
 'BT

Достаю все доступные значения по опциону:

In [5]:
t1=dt.datetime.now().timestamp()
a=[]
key_info='https://api-testnet.bybit.com/option/usdc/openapi/public/v1/tick?symbol='

for k in symbols:
    a.append(rq.get(key_info+k).json()['result'])
t2 = dt.datetime.now().timestamp()

print('Processing time:', t2 - t1) #100-200 секунд (зависит от интернета)

Processing time: 124.61990880966187


In [6]:
df=pd.json_normalize(a) #привожу к формату dataframe

In [7]:
#Удаляю строки с пустыми значениями:
df.replace('',np.nan,inplace=True)
df.dropna(subset='symbol', inplace=True)

#оставляю только нужные столбцы:
df2=df[['symbol','bid','ask','markPrice','markPriceIv']]

In [8]:
df2[['asset','contract_date','strike','option_type']]=df2['symbol'].str.split('-',expand=True)
df2.drop('symbol', axis=1,inplace=True)
df2.set_index('strike',inplace=True)

C:\Users\densm\AppData\Local\Temp\ipykernel_7832\1802089552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['asset','contract_date','strike','option_type']]=df2['symbol'].str.split('-',expand=True)
C:\Users\densm\AppData\Local\Temp\ipykernel_7832\1802089552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['asset','contract_date','strike','option_type']]=df2['symbol'].str.split('-',expand=True)
C:\Users\densm\AppData\Local\Temp\ipykernel_7832\1802089552.py:1: SettingWithCopyWarning: 
A value

In [10]:
df2.head(10)

,bid,ask,markPrice,markPriceIv,asset,contract_date,option_type
strike,,,,,,,
300000,5,0,0.10031766,1.1221,BTC,30DEC22,C
300000,0,0,278482.81031766,1.1221,BTC,30DEC22,P
250000,0,0,0.21006637,1.0921,BTC,30DEC22,C
250000,0,0,228482.93006538,1.0921,BTC,30DEC22,P
200000,20,0,0.51219044,1.0535,BTC,30DEC22,C
200000,0,0,178483.23219044,1.0535,BTC,30DEC22,P
150000,0,0,1.5942029,1.0008,BTC,30DEC22,C
150000,0,0,128484.67399444,1.0008,BTC,30DEC22,P
120000,0,0,3.83172775,0.9578,BTC,30DEC22,C
